In [3]:
!pip install transformers

#### Tokenizer and linguistic model download

In [4]:
from transformers import BertTokenizer, BertForMaskedLM
import torch

tokenizer = BertTokenizer.from_pretrained('miemBertProject/miem-scibert-linguistic')
model = BertForMaskedLM.from_pretrained('miemBertProject/miem-scibert-linguistic')

#### Set input sentence for model and mask token #11 ("риса")

In [5]:
input_ids = tokenizer("Сравнительное влияние климата и обновления культивации на урожайность риса в Китае", return_tensors="pt")["input_ids"]
idx_to_predict=11
input_ids[0,idx_to_predict]=tokenizer.mask_token_id

####Pass through the model to predict masked token

In [6]:
with torch.no_grad():
    outputs = model(input_ids, labels=input_ids)
    loss, prediction_scores = outputs[:2]
predicted_index = torch.argmax(prediction_scores[0][idx_to_predict]).item()
predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])

####Predicted token

In [7]:
predicted_token,tokenizer.convert_ids_to_tokens([tokenizer("Сравнительное влияние климата и обновления культивации на урожайность риса в Китае", return_tensors="pt")["input_ids"][0,idx_to_predict]])

(['риса'], ['риса'])

####Get text embedding via miem-scibert-linguistic

In [8]:
from transformers import BertModel
model=BertModel.from_pretrained('miemBertProject/miem-scibert-linguistic')
model.eval()
input_ids = tokenizer("Сравнительное влияние климата и обновления культивации на урожайность риса в Китае", return_tensors="pt")["input_ids"]
outputs=model(input_ids)

Some weights of the model checkpoint at miemBertProject/miem-scibert-linguistic were not used when initializing BertModel: ['cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at miemBertProject/miem-scibert-linguistic and are newly initialized: ['bert.pooler.dens

In [9]:
outputs[0].shape, outputs[1].shape

(torch.Size([1, 15, 768]), torch.Size([1, 768]))

In [ ]:
outputs[0]~[Number of texts in batch; number of tokens in text; hidden size=768] Tokens embeddings
outputs[1]~[Number of texts in batch; hidden size=768] Text embedding 

In [ ]:
outputs[0]~[Количество текстов в баче; Количество токенов в тексте; Размер скрытого пространства=768] Эмбеддинги токенов
outputs[1]~[Количество текстов в баче; Размер скрытого пространства=768] Эмбеддинг текста

#### Dataset example for RGNTI-classification task

In [14]:
import pandas as pd

df = pd.read_csv('subsample_texts_short.csv') #you should upload csv-file to env in case colab usage 
df.head()

,Unnamed: 0,Unnamed: 0.1,title,ref_txt,kw_list
0,0,301268,Супрамолекулярные системы на основе полиэтилен...,Методами тензиометрии и динамического рассеяни...,агрегация\каликсрезорцины\каталитические свойс...
1,1,209251,Рейтинговые оценки зарубежных авиакосмических ...,Рассмотрена возможность применения матричного ...,Россия\авиакосмическая отрасль\авиастроение\де...
2,2,278388,Применение модифицированного связующего для из...,Рассмотрены свойства и режимы прессования древ...,и карбамидоформальдегидные как основа модифици...
3,3,301787,Зависимость состава и обилия видов растений от...,По данным полевых исследований 1996 2010 гг в...,леса сосновые лесные почвы растительность лесн...
4,4,255697,Российский и международный опыт реализации про...,Проблема выявления врожденной тугоухости не те...,аудиологический скрининг врожденная тугоухость...
